In [1]:
from random import randint
from copy import copy
from collections import defaultdict
from tkinter import Tk, Frame, Button, Label, Text, Scrollbar, Entry

In [2]:
class Message:
    def __init__(self):
        self.day = 0
        self.step = 0
        self.bills = []
        self.check_outs = []

    def add_bill(self, uid, room_type, people, time_before, time_to):
        self.bills.append("id %d %s booked for %d people in %dd for %dd\n" % (uid,
                                                                              room_type,
                                                                              people,
                                                                              time_before,
                                                                              time_to))

    def add_check_out(self, uid, room_type, people, cost):
        self.check_outs.append("id %d check-out from %s for %d with cost %d\n" % (uid,
                                                                                  room_type,
                                                                                  people,
                                                                                  cost))

    def print_bills(self):
        tmp = "Step %d\nDay %d" % (self.day, self.step)
        for bill in self.bills:
            tmp += bill
        return tmp

    def print_check(self):
        tmp = "Step %d\nDay %d" % (self.day, self.step)
        for check in self.check_outs:
            tmp += check
        return tmp

In [3]:
class Room:
    def __init__(self, class_, price, cap):
        # степень комфорта (тип) номера
        self.class_ = class_
        # стоимость номера
        self.price = price
        # местность номера
        self.cap = cap
        # заявки пришедшие на этот номер
        self.app_array = []

In [4]:
class Application:
    def __init__(self, uid, app_type, room_type, people, time_to_book, time_before_booking=0):
        # тип заявки
        self.type = app_type
        # тип номера в заявке
        self.room_type = room_type
        # число человек на заселение
        self.people = people
        # время бронирования/заселения
        self.time_to_book = time_to_book
        # время до заселения (если заявка на бронь)
        self.time_before_booking = time_before_booking
        # уникальный номер заявки
        self.uid = uid

In [5]:
class EventGenerator:
    # конструктор
    def __init__(self, app_types, room_types, room_cap):
        # типы заявок
        self.app_types = app_types
        # типы номеров
        self.room_types = room_types
        # местность номеров
        self.room_cap = room_cap
        # максимальное время для брони
        self.max_time_to_book = 7
        # максимально время до брони
        self.max_time_before_booking = 7
        # номер заявки
        self.app_id = -1

    # создание новой заявки
    def get_new_application(self):
        app_type = randint(1, len(self.app_types))
        room_type = randint(1, len(self.room_types))
        people = randint(1, len(self.room_cap))

        time_to_book = randint(1, self.max_time_to_book)
        time_before_booking = randint(
            1, self.max_time_before_booking) if self.app_types[app_type] == "booking" else 0
        self.app_id += 1

        return Application(self.app_id, app_type, room_type, people, time_to_book, time_before_booking)

In [6]:
class Hotel:
    # конструктор
    def __init__(self, n_rooms, app_types, room_types, room_cap):
        # номера гостницы
        self.rooms = self.__generate_rooms__(n_rooms)
        # выручка
        self.profit = 0
        # типы заявок
        self.app_types = app_types
        # типы номеров
        self.room_types = room_types
        # местность номеров
        self.room_cap = room_cap
        # список поступивших заявок
        self.all_apps = defaultdict()
        # число отклоненных заявок
        self.rejected = 0

    # заполнение гостиницы номерами
    def __generate_rooms__(self, n_rooms):
        return {
            "standart_1":       self.__compute_ratio__("standart", 100, 1, n_rooms, 50, 25),
            "standart_2":       self.__compute_ratio__("standart", 120, 2, n_rooms, 50, 50),
            "standart_3":       self.__compute_ratio__("standart", 140, 3, n_rooms, 50, 25),
            "junior_suite_1":   self.__compute_ratio__("junior_suite", 150, 1, n_rooms, 30, 25),
            "junior_suite_2":   self.__compute_ratio__("junior_suite", 180, 2, n_rooms, 30, 50),
            "junior_suite_3":   self.__compute_ratio__("junior_suite", 210, 3, n_rooms, 30, 25),
            "suite_1":          self.__compute_ratio__("suite", 200, 1, n_rooms, 20, 25),
            "suite_2":          self.__compute_ratio__("suite", 240, 2, n_rooms, 20, 50),
            "suite_3":          self.__compute_ratio__("suite", 280, 3, n_rooms, 20, 25),
        }

    def __compute_ratio__(self, class_, price, cap, n_rooms, ratio1, ratio2):
        ratio = n_rooms*ratio1//100*ratio2//100
        return [Room(class_, price, cap) for i in range(ratio)]

    # обработка заявки
    def process_application(self, application):
        res = ""

        room_type = self.room_types[application.room_type] + \
            "_"+str(application.people)
        for room_idx, room in enumerate(self.rooms[room_type]):
            found_room = 1
            for app in room.app_array:
                if set(range(app.time_before_booking, app.time_to_book)) & set(range(application.time_before_booking*24, application.time_to_book*24)):
                    found_room = 0
                    break

            if found_room:
                self.all_apps[application.uid] = copy(application)
                res += "id %d %s booked for %d people in %dd for %dd\n" % (application.uid,
                                                                           self.room_types[application.room_type],
                                                                           application.people,
                                                                           application.time_before_booking,                                                                            application.time_to_book)
                application.time_before_booking *= 24
                application.time_to_book *= 24
                self.rooms[room_type][room_idx].app_array.append(application)
                return res

        res += "No suitable room for %s for %d people from %d for %d\n" % (self.room_types[application.room_type],
                                                                           application.people,
                                                                           application.time_before_booking,
                                                                           application.time_to_book)
        self.rejected += 1
        return res

    # обновление времени
    def process_one_tick(self):
        res = ""

        for key in self.rooms:
            for room_idx, room in enumerate(self.rooms[key]):
                for app_idx, app in enumerate(room.app_array):
                    if app.time_before_booking == 0:
                        if app.time_to_book == 0:
                            cost = room.price * \
                                self.all_apps[app.uid].time_to_book
                            self.profit += cost
                            res += "id %d check-out from %s for %d with cost %d\n" % (app.uid,
                                                                                      self.room_types[app.type],
                                                                                      app.people,
                                                                                      cost)
                            self.rooms[key][room_idx].app_array.remove(app)
                        else:
                            self.rooms[key][room_idx].app_array[app_idx].time_to_book -= 1
                    else:
                        self.rooms[key][room_idx].app_array[app_idx].time_before_booking -= 1
        return res

    # получение данных о занятости номеров
    def get_data(self, day, max_days, data):
        new = False
        if len(data) == 0:
            new = True

        idx = 0
        for key in self.rooms:
            for room in self.rooms[key]:
                if new:
                    days = [-1]*max_days
                else:
                    days = data[idx]

                for app in room.app_array:
                    if app.time_before_booking == 0:
                        days[day] = 2
                        for i in range(day+1, min(day+app.time_to_book//24+1, max_days)):
                            days[i] = 0
                    else:
                        for i in range(min(day+app.time_before_booking//24+1, max_days), min(day+app.time_before_booking//24+1+app.time_to_book//24+1, max_days)):
                            days[i] = 1

                if new:
                    data.append(days)
                else:
                    data[idx] = days
                idx += 1
        return data

    # получение данных о состоянии гостиницы
    def report(self):
        return self.profit, len(self.all_apps), self.rejected

In [7]:
class Experiment:
    # конструктор
    def __init__(self, step=5, n_rooms=24, simulation_period=30, max_time_for_new_application=5):
        # шаг
        self.step = step
        # число номеров в гостнице
        self.n_rooms = n_rooms
        # период моделирования
        self.period = simulation_period
        # верхняя граница времени до новой заявки
        self.max_time = max_time_for_new_application
        # текущее время эксперимента
        self.curr_time = 0
        # текущий день эксперимента
        self.curr_day = 0
        # текущее время дня
        self.time = 0
        # время до поступления заявки
        self.time_for_new_application = -1
        # данные о занятости номеров
        self.data = []

        app_types = {1: "booking", 2: "checking-in"}
        room_types = {1: "standart", 2: "suite", 3: "junior_suite"}
        room_cap = [1, 2, 3]

        # генератор событий
        self.event_generator = EventGenerator(app_types, room_types, room_cap)
        # гостиница
        self.hotel = Hotel(self.n_rooms, app_types, room_types, room_cap)
        # сообщение о подтверждении брони
        self.message = ""

    # один шаг эксперимента
    def make_step(self):
        self.bills = ""
        self.check = ""

        if self.time == self.period * 24:
            return self.bills, self.check, self.data, self.hotel.report()

        self.bills += "Step %d\n" % (self.time//self.step)
        self.check += "Step %d\n" % (self.time//self.step)

        for _ in range(self.step):

            if self.time % 24 == 0:
                self.bills += "Day %d\n" % (self.time//24)
                self.check += "Day %d\n" % (self.time//24)

            self.curr_time = self.time % 24
            self.curr_day = self.time // 24

            if self.time_for_new_application < 0:
                self.time_for_new_application = randint(1, self.max_time)
            elif self.time_for_new_application == 0:
                application = self.event_generator.get_new_application()

                self.bills += self.hotel.process_application(application)
                self.time_for_new_application = randint(1, self.max_time)
            else:
                self.time_for_new_application -= 1

            self.hotel.get_data(self.curr_day, self.period, self.data)
            self.check += self.hotel.process_one_tick()

            self.time += 1

        return self.bills, self.check, self.data, self.hotel.report()

    # провести эксперимент до конца
    def complete(self):
        full_in = ""
        full_out = ""
        while True:
            in_, out_, data, report = self.make_step()
            if len(in_) and len(out_):
                full_in += in_
                full_out += out_
            else:
                full_in += "FINISH\n"
                return full_in, full_out, data, report

In [ ]:
def draw(in_, out_, report, data):

    check_in.text.insert("end", in_)
    check_in.text.see("end")

    check_out.text.insert("end", out_)
    check_out.text.see("end")

    total.total['text'] = "Total income: %d" % report[0]
    total.proc['text'] = "Total applications proceeded: %d" % report[1]
    total.rej['text'] = "Total applications rejected: %d" % report[2]

    if len(data) != 0:
        for idx, row in enumerate(table._widgets):
            for jdx, _ in enumerate(row):
                if data[idx][jdx] == 0:
                    table._widgets[idx][jdx]['bg'] = "red"
                elif data[idx][jdx] == 1:
                    table._widgets[idx][jdx]['bg'] = "yellow"
                elif data[idx][jdx] == 2:
                    table._widgets[idx][jdx]['bg'] = "blue"
                else:
                    table._widgets[idx][jdx]['bg'] = "green"


def step():
    complete = False
    in_, out_, data, report = experiment.make_step()

    if len(in_) == 0 and len(out_) == 0:
        complete = True
        in_, out_,  data, report = experiment.complete()

    draw(in_, out_, report, data)

    if complete:
        buttons.stepb['state'] = 'disabled'
        buttons.runb['state'] = 'disabled'


def run():
    in_, out_, data, report = experiment.complete()

    draw(in_, out_, report, data)

    buttons.stepb['state'] = 'disabled'
    buttons.runb['state'] = 'disabled'

def init():
        global experiment
        global table
        
        n_rooms = int(total.K.get())
        simulation_period=int(total.M.get())
        
        experiment = Experiment(n_rooms=n_rooms, simulation_period=simulation_period)
        
        table.destroy()
        table = SimpleTable(root)
        table.grid(row=3, column=1, columnspan=2)
        
        buttons.init['state'] = 'disabled'


class Buttons(Frame):
    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)
        self.stepb = Button(self, text='step',
                            command=step, height=5, width=15)
        self.runb = Button(self, text='run', command=run, height=5, width=15)
        self.init = Button(self, text='init', command=init, height=5, width=15)
        
        self.stepb.grid(row=1, column=1, padx=(20, 20))
        self.runb.grid(row=1, column=2, padx=(20, 20))
        self.init.grid(row=1, column=3, padx=(20,20))
        

class Total(Frame):
    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)
        self.total = Label(self, text="Total income: 0", width=30, anchor='w')
        self.proc = Label(
            self, text="Total applications proceeded: 0", width=30, anchor='w')
        self.rej = Label(
            self, text="Total applications rejected: 0", width=30, anchor='w')

        self.total.grid(row=0, column=0, padx=(20, 20))
        self.proc.grid(row=1, column=0, padx=(20, 20))
        self.rej.grid(row=2, column=0, padx=(20, 20))
        
        self.M_text = Label(self, text="M days = ", width=10)
        self.K_text = Label(self, text="K rooms = ", width=10)
        self.M = Entry(self, width=10)
        self.K = Entry(self, width=10)
        
        self.M_text.grid(row=0, column=1)
        self.K_text.grid(row=1, column=1)
        self.M.grid(row=0, column=2)
        self.K.grid(row=1, column=2)


class CheckIn(Frame):
    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)

        self.text = Text(self, height=20, width=60)
        self.vsb = Scrollbar(self, orient="vertical", command=self.text.yview)
        self.text.configure(yscrollcommand=self.vsb.set)
        self.vsb.pack(side="right", fill="y")
        self.text.pack(side="left", fill="both", expand=True)


class CheckOut(Frame):
    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)

        self.text = Text(self, height=20, width=60)
        self.vsb = Scrollbar(self, orient="vertical", command=self.text.yview)
        self.text.configure(yscrollcommand=self.vsb.set)
        self.vsb.pack(side="right", fill="y")
        self.text.pack(side="left", fill="both", expand=True)


class SimpleTable(Frame):
    def __init__(self, parent):
        Frame.__init__(self, parent, background="black")
        self._widgets = []

        label = Label(self, text="Rom type / Day",
                      borderwidth=0, width=15)
        label.grid(row=0, column=0, sticky="nsew", padx=1, pady=1)

        for column in range(experiment.period):
            label = Label(self, text="%d" % column,
                          borderwidth=0, width=4)
            label.grid(row=0, column=column+1, sticky="nsew", padx=1, pady=1)

        row_num = 1
        for room_name in experiment.hotel.rooms:
            for room in experiment.hotel.rooms[room_name]:
                current_row = []
                label = Label(self, text="%s" % room_name,
                              borderwidth=0, width=15)
                label.grid(row=row_num, column=0,
                           sticky="nsew", padx=1, pady=1)

                for column in range(experiment.period):
                    label = Label(self, text="-",
                                  borderwidth=0, width=4)
                    label.grid(row=row_num, column=column+1,
                               sticky="nsew", padx=1, pady=1)
                    current_row.append(label)
                self._widgets.append(current_row)
                row_num += 1


experiment = Experiment()

root = Tk()
buttons = Buttons(root)
total = Total(root)
check_in = CheckIn(root)
check_out = CheckOut(root)
table = SimpleTable(root)

buttons.grid(row=1, column=1, sticky="w")
total.grid(row=1, column=2, sticky="w")
check_in.grid(row=2, column=1)
check_out.grid(row=2, column=2)
table.grid(row=3, column=1, columnspan=2)

root.minsize(1000, 600)
root.resizable(False, False)
root.mainloop()